In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from pandas.tseries.offsets import BDay

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML



import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Only Cont Fut Flag
only_cont_fut = False

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    if only_cont_fut == True:   
        if 'Fut' in exo_dict['name']:   
            exo_series = exo_series[exo_series.columns[0]]
            df[exo_dict['name']] = exo_series
            
    if only_cont_fut == False:
        exo_series = exo_series[exo_series.columns[0]]
        df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.head()

In [ ]:
px_ser = df.ZW_SmartEXO_Bullish_Ichi_Short_Put_Spread_150Delta_Bi.dropna()

In [ ]:
forward_line = px_ser.rolling(30, center=True, win_type='barthann').mean().shift(30)
forward_line.index += BDay(15)

px_ser.plot()
forward_line.plot()

#px_ser.rolling(29, center=True, win_type='barthann').mean().shift(29).plot()
#px_ser.rolling(100, center=True).mean().shift(100).plot()

In [ ]:
px_ser.plot()

px_ser.rolling(30, center=True, win_type='barthann').mean().shift(30).plot()
px_ser.rolling(30, center=False).mean().shift(0).plot()

In [ ]:
px_ser.plot()
px_ser.rolling(30, center=True, win_type='barthann').mean().shift(30).plot()

#px_ser.rolling(30*10, center=True, win_type='barthann').mean().plot()

(px_ser - px_ser.rolling(30, center=True, win_type='barthann').mean().shift(30)).plot(secondary_y=True)

In [ ]:
px_diff = px_ser.rolling(20, win_type='hamming').mean().diff()
#px_diff = px_ser.diff()

threshold_diff_ser = pd.Series(0.0, index=px_diff.index)

threshold_value = 1

for i in range(px_ser.size):
    if px_diff[i] > 0:
        if px_diff[i] > threshold_value:
            threshold_diff_ser[i] = threshold_value
            
        else:
            threshold_diff_ser[i] = px_diff[i]
            
    if px_diff[i] < 0:
        if px_diff[i] < -threshold_value:
            threshold_diff_ser[i] = -threshold_value
            
        else:
            threshold_diff_ser[i] = px_diff[i]

In [ ]:
px_ser.plot()
px_ser.rolling(20, win_type='hamming').mean().plot()

threshold_diff_ser.cumsum().plot(secondary_y=True)
#(threshold_diff_ser.cumsum() - threshold_diff_ser.cumsum().rolling(20).mean()).plot()
#axhline(0)

In [ ]:
threshold_px_ser = threshold_diff_ser.cumsum()

threshold_px_ser.plot()
threshold_px_ser.rolling(20).median().plot()

In [ ]:
threshold_px_ser.diff(10).plot()